In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sys
import warnings
sys.path.append('..')
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import yaml

In [3]:
X_train = pd.read_csv('dataset/X_train.csv', index_col=0, parse_dates=[0])
y_train = pd.read_csv('dataset/y_train.csv', index_col=0, parse_dates=[0])
X_test = pd.read_csv('dataset/X_test.csv', index_col=0, parse_dates=[0])
y_test = pd.read_csv('dataset/y_test.csv', index_col=0, parse_dates=[0])

In [4]:
with open('../config/config.yaml', 'r') as file:
    data_loaded = yaml.safe_load(file)

In [15]:
price = pd.read_csv('../db/raw.csv', index_col=0, header=[0,1], parse_dates=[0])
price_data = price.loc[:, pd.IndexSlice[['Close'], ['^GDAXI']]]
price_data.columns = ['gdaxi']

In [22]:
data_loaded.keys()

dict_keys(['selected_features', 'threshold', 'xgb_classifier_params'])

In [24]:
model = xgb.XGBClassifier(**data_loaded['xgb_classifier_params'])

In [26]:
model.fit(X_train[data_loaded['selected_features']], y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.827531045059713, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.07503077283716637,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0021561608426896948,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=17, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=932, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [19]:
X_test.index

DatetimeIndex(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04',
               '2024-01-05', '2024-01-08', '2024-01-09', '2024-01-10',
               '2024-01-11', '2024-01-12',
               ...
               '2024-05-13', '2024-05-14', '2024-05-15', '2024-05-16',
               '2024-05-17', '2024-05-20', '2024-05-21', '2024-05-22',
               '2024-05-23', '2024-05-24'],
              dtype='datetime64[ns]', length=105, freq=None)

In [56]:
price_data.loc[idx, 'gdaxi']
# price_data

KeyError: Timestamp('2024-01-01 00:00:00')

In [43]:
initial_cash = 1000000
cash = initial_cash
allocation_size = .1
investment = list()
investment_minimum = 10000

for idx, row  in X_test.iterrows():
    input_data = row[data_loaded['selected_features']].to_frame().T
    to_buy = (model.predict_proba(input_data)[:, 1] > data_loaded['threshold'])[0]
    
    if to_buy:
        
        if cash * allocation_size < investment_minimum:
            continue
        
        to_invest_capital = int(cash * allocation_size)
        cash = cash - to_invest_capital
    
    print(idx, prediction)
    
    break

# cash = initial_cash
# positions = []
# portfolio = []

# for current_date in data.index:
#     # Check for a buy signal
#     if data.loc[current_date, 'signal'] == 1:
#         buy_price = data.loc[current_date, 'price']
#         positions.append((current_date, buy_price))
#         cash -= buy_price
    
#     # Check if any positions need to be sold (10 days later)
#     positions_to_sell = [p for p in positions if (current_date - p[0]).days >= 10]
    
#     for position in positions_to_sell:
#         sell_date, buy_price = position
#         sell_price = data.loc[current_date, 'price']
#         cash += sell_price
#         portfolio.append((sell_date, current_date, buy_price, sell_price))
#         positions.remove(position)

# # Calculate the final value of the portfolio
# final_cash = cash + sum([data.loc[data.index[-1], 'price'] for _, buy_price in positions])
# profit = final_cash - initial_cash

# # Output results
# portfolio_df = pd.DataFrame(portfolio, columns=['Buy Date', 'Sell Date', 'Buy Price', 'Sell Price'])
# print("Final Portfolio Value:", final_cash)
# print("Total Profit:", profit)
# print("\nTrade Details:\n", portfolio_df)


2024-01-01 00:00:00 False
